## BERT系でベースラインを組む
* 実験管理ちゃんとしよう


In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import MeCab
import re
import demoji, mojimoji
import neologdn

from config import *
from myutils import *

In [24]:
train = pd.read_csv(data_path+"train.csv")
test = pd.read_csv(data_path+"test.csv")

df = pd.concat([train, test]).reset_index(drop=True)
df.head(2)

,id,source,text,label
0,80074aa43,news4vip,まともに相手されてない人との関係なんて\nそんな大事にするものか？,0.0
1,6378fea6b,livejupiter,最近はアヘアヘQSマンやない？ ｲｲ!(・∀・)+1-0(・Ａ・)ｲｸﾅｲ!,0.0


In [25]:
df["clean_text"] = df["text"].map(lambda x: clean_text(x))

### BERTの実装 --

In [26]:
cfg = {
    "model_name": "cl-tohoku/bert-base-japanese-whole-word-masking",
    "max_length": -1,
    "train_epoch": 2,
    "lr": 3e-5,
}

In [27]:
from transformers import BertJapaneseTokenizer, BertForSequenceClassification
tokenizer = BertJapaneseTokenizer.from_pretrained(
    cfg["model_name"],
    mecab_kwargs={"mecab_dic":None, "mecab_option": f"-d {dic_neologd}"}
)
model = BertForSequenceClassification.from_pretrained(cfg["model_name"])

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

In [28]:
tokenizer.encode(df["clean_text"][0])

[2,
 23135,
 7,
 1879,
 26,
 20,
 16,
 18,
 21,
 28514,
 13,
 5,
 633,
 15060,
 4799,
 15872,
 7,
 34,
 120,
 29,
 2935,
 3]

In [29]:
tokenizer.tokenize("鬼滅の刃ガッキー逃げ恥")

['鬼', '##滅', '##の', '##刃', 'ガ', '##ッキー', '逃げ', '##恥']

In [30]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize("鬼滅の刃ガッキー逃げ恥"))

[4262, 29486, 28444, 30558, 444, 3660, 4738, 30840]